# CHEM 1000 - Fall 2020
Prof. Geoffrey Hutchison, University of Pittsburgh

## 6 Optimizing Functions

Chapter 6 in [*Mathematical Methods for Chemists*](http://sites.bu.edu/straub/mathematical-methods-for-molecular-science/)

By the end of this session, you should be able to:
- Understand general approaches to optimize functions either for minima or maxima (i.e. "extrema")
- Understand how to use derivatives in multiple dimensions to categorize extrema

### Why

In chemistry and physics, we often want to determine the maximum or minimum value of a function of one or many variables. Examples include characterizing the minima, maxima, and saddle points on a potential energy surface.

Remember from Calculus, that at a maximum or a minimum, the derivative must be zero.

<img src="../images/extrema.png" />

To decide on the type of extrema, we look at the derivative:

$$
\left.\frac{d^{2} f(x)}{d x^{2}}\right|_{x=x^{*}}=\left\{\begin{array}{ll}
<0, & \text { maximum } \\
=0, & \text { inflection point } \\
>0, & \text { minimum }
\end{array}\right.
$$

In other words, the second derivative is negative at a maxima because it "points down." At a minima, the second derivative is positive and "points up." At an inflection point, the derivative isn't zero, but the second derivative is, because on one side it is positive and on the other side it is negative.

Thus, optimizing functions in one dimension is pretty easy, if sometimes tedious.
1. Take derivatives and find where the first derivative is zero
2. Look at the second derivatives, to categorize as a minima / maxima / inflection point
3. Then compare values of the function at those points to see if it's a local minima / max or the global minima / max.

### Many Variables

Not surprisingly, we can use a similar technique in multiple dimensions.

If we have a function $f(x,y)$ in two dimensions, then to have an extrema:

$$
\frac{\partial f}{\partial x}=0 \quad \frac{\partial f}{\partial y}=0
$$

In other words, we need to see the partial derivative with respect to **both / all** variables be zero.

We can then categorize the type of minima / maxima with the [Hessian]. (Later, we will see that this is the *determinant* of the Hessian matrix, for when we have more than 2 variables.)

$$
D=\left(\frac{\partial^{2} f}{\partial x^{2}}\right)\left(\frac{\partial^{2} f}{\partial y^{2}}\right)-\left(\frac{\partial^{2} f}{\partial x \partial y}\right)\left(\frac{\partial^{2} f}{\partial y \partial x}\right)
$$


$$
\left.\frac{\partial^{2} f}{\partial x^{2}}\right|_{\left(x^{*}, y^{*}\right)}=\left\{\begin{array}{ll}
<0 & D>0 & \text { maximum } \\
>0 & D>0 & \text { minimum } \\
 & D < 0 & \text { saddle-point }
\end{array}\right.
$$

### Example:

Let's try the example of a potential energy surface (i.e., a double-well potential):

$$
V(x, y)=\left(x^{2}-1\right)^{2}+y^{2}
$$

<img src='../images/saddle.png' />

In [1]:
from sympy import init_session
init_session()

IPython console for SymPy 1.6.2 (Python 3.6.11-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.6.2/



In [2]:
V = (x**2 - 1)**2 + y**2

In [4]:
# okay, let's see where the partial derivative with x is zero...
diff(V, x)

In [5]:
# we can use the Sympy solve() function
# although that one isn't very hard
solve( diff(V, x) )

In [6]:
# now let's do y
diff(V, y)

Okay, so for this potential energy function, we have three zero points:
- (-1, 0)
- (0, 0)
- (+1, 0)

What kind of points are these?

In [7]:
# here's the two-variable Hessian test...
D = diff(V, x, 2)*diff(V, y, 2) - diff(V, x, y)*diff(V, y, x)
D # print it

In [8]:
# we can use the .subs() method to substitute values (-1, 0)
D.subs([ (x, -1), (y, 0) ])

In [9]:
# check (0,0)
D.subs([ (x, 0), (y, 0) ])

In [10]:
# now check (+1, 0)
D.subs([ (x, 1), (y, 0) ])

To recap:
- (-1, 0): D is positive, second derivative is positive, this is a minima
- (0, 0): D is negative, this must be a saddle point
- (+1, 0): D is positive, second derivative is positive, this is a minima

So we can use the Hessian D to establish the type of minima / maxima / saddle point.

### More Complicated Functions

Sometimes in chemistry, functions are complicated enough that we can't easily find all the zeros.

When?

Let's imagine I want to optimize the geometry of a molecule. Each atom can move independently of the other atoms, so that's:
- X, Y, and Z displacement for atom 1
- X, Y, and Z displacement for atom 2
- .. etc.

So for N atoms, that's 3 displacements per atom, or 3N. To be precise, I'll need to subtract out cases where I move all the atoms the same amount in the X, Y, or Z directions - that's just moving the whole molecule. And I'll need to subtract our cases where I rotate the whole molecule along the X, Y, or Z axis.. so generally I have 3N-6 variabels to optimize. For water, that's 3 variables, but even for ethane, that's 8 atoms and 18 variables to optimize at once.

How do we optimize ethane or something more complicated?

<img src="../images/atom-forces.png" width="341" />

Since we can presumably calculate the potential energy as a function of the atom positions, we calculate:

$$
V(1, 2, 3 .. 8)
$$

Then we calculate the gradients on each atom:

$$
\boldsymbol{F} = -\boldsymbol{\nabla} V
$$

We push the atoms a little bit, and repeat. How much do we push the atoms? There are a few methods:
- [steepest gradient descent](https://en.wikipedia.org/wiki/Gradient_descent)
- [conjugate gradient](https://en.wikipedia.org/wiki/Conjugate_gradient_method)
- [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) or limited-memory L-BFGS

In some sense these are all methods that determine how much you should move along the gradient, and then what should you do .. to eventually end up at the extrema. (You pick whether you're going uphill or downhill to Point State Park.)

### Example: 

A standard case for optimizing multi-variate functions is the [Rosenbrock function](https://en.wikipedia.org/wiki/Rosenbrock_function):

$$
f(\mathbf{x})=\sum_{i=1}^{N-1} 100\left(x_{i+1}-x_{i}^{2}\right)^{2}+\left(1-x_{i}\right)^{2}
$$

So you pick some number of dimensions N, and the minimum is clearly when all the $x_i = 1$

In [11]:
import numpy as np
# the scipy.optimize module has the Rosenbrock function
from scipy.optimize import rosen
# it also has a bunch of minimize methods
from scipy.optimize import minimize

# here are our initial values - 5 dimensions
# all somewhat close to 1
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

rosen(x0) # close to 1, but still pretty big

In [13]:
# we can minimize using the Conjugate Gradients method "CG"
# the 'disp' True part will show us some information about how many steps
optima = minimize(rosen, x0, method='CG', options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 67
         Function evaluations: 834
         Gradient evaluations: 139


In [16]:
# the variable result has a bunch of things in it
dir(optima)

['fun', 'jac', 'message', 'nfev', 'nit', 'njev', 'status', 'success', 'x']

In [18]:
# the most useful is 'x' - the array of minimized values
print(optima.x)
# also useful is 'fun' - the value of the function at optima.x
# e.g., rosen(optima.x)
print(optima.fun)

[0.99999927 0.99999853 0.99999706 0.99999411 0.99998819]
4.610989257647561e-11
4.610989257647561e-11


In short, we can use the Conjugate Gradients method to optimize the function - it took 67 steps and 834 evaluations of the Rosenbrock function. Are there better algorithms?

In [19]:
# Let's try BFGS instead of "CG"
optima = minimize(rosen, x0, method='BFGS', options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 25
         Function evaluations: 180
         Gradient evaluations: 30


In [20]:
# let's see the optimized x values and rosen(optima.x)
print(optima.x)
print(optima.fun)

[0.99999925 0.99999852 0.99999706 0.99999416 0.99998833]
4.581718348719802e-11


So the BFGS method takes fewer steps then conjugate gradients (25 vs. 67) and fewer function evaluations (180 vs. 834).

If we had a more complicated function, a slower computer, .. maybe we're doing quantum mechanics and calculating the energy of the molecule takes time.. clearly we want to use efficient optimization methods. BFGS and the L-BFGS method are pretty good if you have can easily calculate the gradient.

Now if you can also easily calculate the Hessian, you can use another method - a modified Newton's method "Newton-CG"

In [21]:
# here, we import the known derivative and Hessian of the Rosenbrock function
# - the Gradient (rosen_der)
# - the Hessian (rosen_hess)
from scipy.optimize import rosen_der, rosen_hess

# Newton-CG is a little more complicated
# because we have to give these too
optima = minimize(rosen, x0, method='Newton-CG',
               jac=rosen_der, hess=rosen_hess,
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 21
         Function evaluations: 30
         Gradient evaluations: 30
         Hessian evaluations: 21


To summarize:
- Conjugate Gradients (only need gradient, 67 steps, 834 function calls)
- BFGS (also only need gradient, 25 steps, 180 function calls)
- Newton-CG (gradient and Hessian, 21 steps, 30 function calls)

In short, when we have really complicated functions, we optimize numerically using one of these methods, ideally BFGS or a Newton-CG method depending on  whether we have the gradient and/or Hessian to compute.

Beyond these techniques, there is a whole area of mathematics on "[optimization theory](https://en.wikipedia.org/wiki/Mathematical_optimization)"

Gradient-free techniques include:
- [Particle swarm optimization](https://en.wikipedia.org/wiki/Particle_swarm_optimization)
- [Simultaneous perturbation stochastic approximation](https://www.jhuapl.edu/SPSA/)
- [Genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) - also work with discrete or non-continuous functions
- [Simulated annealing](https://en.wikipedia.org/wiki/Simulated_annealing)
- [Simplex / Nelder-Mead](https://en.wikipedia.org/wiki/Nelder–Mead_method)

Many of these gradient-free techniques involve estimating a [*surrogate function*](https://en.wikipedia.org/wiki/Surrogate_model) -- a simpler system to be optimized, e.g. [Bayesian optimization / Gaussian processs regression](https://en.wikipedia.org/wiki/Bayesian_optimization). (Some texts call this ["kriging"](https://en.wikipedia.org/wiki/Kriging).)

**Multiple Objectives**

Some cases, you may want to optimize more than one property that have a trade-off. This is known as [Pareto optimization](https://en.wikipedia.org/wiki/Pareto_efficiency), for example fuel efficiency and engine horsepower in a car. 

In quantum chemistry, there is often a trade-off between accuracy and speed. You may make approximations that result in a faster but less accurate method (e.g., you approximate or ignore certain time-consuming integrals). Or you may want a higher accuracy calculation, but that means it will take longer.

-------
This notebook is from Prof. Geoffrey Hutchison, University of Pittsburgh
https://github.com/ghutchis/chem1000

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>